#Install dan import library

In [ ]:
!pip install pyspark findspark
!pip install ipywidgets
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('AnimeAnalysis').getOrCreate()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00


In [ ]:
from pyspark.sql.functions import col, udf, explode, trim
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, udf, explode, trim
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

#Baca dataset

In [ ]:
pandas_df = pd.read_csv('/content/merged_df.csv')
pandas_df

,uid,anime_title,synopsis,genre,aired,episodes,members,ranked,score,img_url,link,Rating,Popularity,Type
0,28891,haikyuu!! second season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,8.62,Oct 2015 - Mar 2016,TV
1,23273,shigatsu wa kimi no uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,8.64,Oct 2014 - Mar 2015,TV
2,34599,made in abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,8.64,Jul 2017 - Sep 2017,TV
3,5114,fullmetal alchemist: brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,9.10,Apr 2009 - Jul 2010,TV
4,31758,kizumonogatari iii: reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,8.78,Jan 2017 - Jan 2017,Movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7348,36043,net-juu no susume special,An unaired 11th episode included with the Blu-...,"['Game', 'Comedy', 'Romance']","Dec 8, 2017",1.0,62448,1802.0,7.48,https://cdn.myanimelist.net/images/anime/7/901...,https://myanimelist.net/anime/36043/Net-juu_no...,7.34,Dec 2017 - Dec 2017,Special
7349,10075,naruto x ut,All-new animation offered throughout UNIQLO cl...,"['Action', 'Comedy', 'Super Power', 'Martial A...","Jan 1, 2011",1.0,34155,1728.0,7.50,https://cdn.myanimelist.net/images/anime/3/304...,https://myanimelist.net/anime/10075/Naruto_x_UT,7.36,Jan 2011 - Jan 2011,OVA
7350,35828,miira no kaikata,High school student Sora Kashiwagi is accustom...,"['Slice of Life', 'Comedy', 'Supernatural']","Jan 12, 2018 to Mar 30, 2018",12.0,61459,1727.0,7.50,https://cdn.myanimelist.net/images/anime/1486/...,https://myanimelist.net/anime/35828/Miira_no_K...,7.46,Jan 2018 - Mar 2018,TV
7351,10378,shinryaku!? ika musume,"After regaining her squid-like abilities, Ika ...","['Slice of Life', 'Comedy', 'Shounen']","Sep 27, 2011 to Dec 25, 2011",12.0,67422,1548.0,7.56,https://cdn.myanimelist.net/images/anime/6/301...,https://myanimelist.net/anime/10378/Shinryaku_...,7.47,Sep 2011 - Dec 2011,TV


In [ ]:
print(pandas_df.isnull().sum())

uid              0
anime_title      0
synopsis       183
genre            0
aired            0
episodes       183
members          0
ranked         184
score            0
img_url         16
link             0
Rating           0
Popularity       0
Type             0
dtype: int64


In [ ]:
spark_df = spark.createDataFrame(pandas_df)
spark_df.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------+-------+------+-----+--------------------+--------------------+------+-------------------+-------+
|  uid|         anime_title|            synopsis|               genre|               aired|episodes|members|ranked|score|             img_url|                link|Rating|         Popularity|   Type|
+-----+--------------------+--------------------+--------------------+--------------------+--------+-------+------+-----+--------------------+--------------------+------+-------------------+-------+
|28891|haikyuu!! second ...|Following their p...|['Comedy', 'Sport...|Oct 4, 2015 to Ma...|    25.0| 489888|  25.0| 8.82|https://cdn.myani...|https://myanimeli...|  8.62|Oct 2015 - Mar 2016|     TV|
|23273|shigatsu wa kimi ...|Music accompanies...|['Drama', 'Music'...|Oct 10, 2014 to M...|    22.0| 995473|  24.0| 8.83|https://cdn.myani...|https://myanimeli...|  8.64|Oct 2014 - Mar 2015|     TV|
|3459

#Pre-processing

In [ ]:
@udf(returnType=ArrayType(StringType()))
def clean_genre(genre_array):
    if genre_array:
        cleaned = str(genre_array).replace('[', '').replace(']', '').replace("'", "").split(',')
        return [g.strip() for g in cleaned]
    return []

df = spark_df.withColumn("clean_genre", clean_genre(col("genre")))
df = df.select('episodes', 'Type', 'score', explode("clean_genre").alias("genre"))
df = df.filter(col("genre").isNotNull() & (trim(col("genre")) != ""))

In [ ]:
genre_indexer = StringIndexer(inputCol="genre", outputCol="genre_index", handleInvalid="skip")
type_indexer = StringIndexer(inputCol="Type", outputCol="type_index", handleInvalid="skip")
genre_encoder = OneHotEncoder(inputCol="genre_index", outputCol="genre_vec")
type_encoder = OneHotEncoder(inputCol="type_index", outputCol="type_vec")

In [ ]:
assembler = VectorAssembler(inputCols=["genre_vec", "type_vec"], outputCol="features")

#Modelling

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='score')
rf = RandomForestRegressor(featuresCol='features',
                           labelCol='score',
                           numTrees=300,
                           )
gbt = GBTRegressor(featuresCol='features',
                   labelCol='score',
                   maxIter=100)

In [ ]:
pipeline_lr = Pipeline(stages=[genre_indexer, type_indexer, genre_encoder, type_encoder, assembler, lr])
pipeline_rf = Pipeline(stages=[genre_indexer, type_indexer, genre_encoder, type_encoder, assembler, rf])
pipeline_gbt = Pipeline(stages=[genre_indexer, type_indexer, genre_encoder, type_encoder, assembler, gbt])

In [ ]:
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
model_lr = pipeline_lr.fit(train_data)
model_rf = pipeline_rf.fit(train_data)
model_gbt = pipeline_gbt.fit(train_data)

#Evaluasi

In [ ]:
predictions_lr = model_lr.transform(test_data)
predictions_rf = model_rf.transform(test_data)
predictions_gbt = model_gbt.transform(test_data)

In [ ]:
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")

lr_rmse = evaluator.evaluate(predictions_lr)
rf_rmse = evaluator.evaluate(predictions_rf)
gbt_rmse = evaluator.evaluate(predictions_gbt)

print(f"Linear Regression RMSE: {lr_rmse}")
print(f"Random Forest RMSE: {rf_rmse}")
print(f"GBT RMSE: {gbt_rmse}")

Linear Regression RMSE: 0.6180643738197574
Random Forest RMSE: 0.622619578351972
GBT RMSE: 0.6184797654381163


#Deployment Sederhana

In [ ]:
# Simpan model yang sudah di-train
model_lr.save("anime_lr_model")
model_rf.save("anime_rf_model")
model_gbt.save("anime_gbt_model")

from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

def create_input_df(spark, genre, type_anime):
    """
    Membuat DataFrame dari input user
    """
    # Schema untuk input
    schema = StructType([
        StructField("genre", ArrayType(StringType()), True),
        StructField("Type", StringType(), True)
    ])

    # DataFrame
    data = [(genre, type_anime)]
    df = spark.createDataFrame([(genre, type_anime)], ["genre", "Type"])

    # Proses genre seperti pada training
    df = df.withColumn("clean_genre", clean_genre(col("genre")))
    df = df.select('Type', explode("clean_genre").alias("genre"))

    return df

def predict_anime_score(genre, type_anime, model_type='lr'):
    """
    Memprediksi skor anime berdasarkan genre dan tipe

    Parameters:
    genre (list): List genre anime (gunakan nama genre sesuai encoding)
    type_anime (str): Tipe anime (TV, Movie, OVA, etc.)
    model_type (str): Tipe model ('lr', 'rf', atau 'gbt')

    Returns:
    float: Prediksi skor anime
    """
    # Load model yang sesuai
    if model_type == 'lr':
        loaded_model = PipelineModel.load("anime_lr_model")
    elif model_type == 'rf':
        loaded_model = PipelineModel.load("anime_rf_model")
    else:
        loaded_model = PipelineModel.load("anime_gbt_model")

    # Buat DataFrame input
    input_df = create_input_df(spark, genre, type_anime)

    # Lakukan prediksi
    predictions = loaded_model.transform(input_df)

    # Ambil rata-rata prediksi (karena bisa ada multiple genre)
    avg_prediction = predictions.select("prediction").agg({"prediction": "avg"}).collect()[0][0]

    return avg_prediction

# Contoh penggunaan
genre_list = ["Action", "Adventure", "Fantasy"]
anime_type = "TV"

print("Linear Regression Prediction:", predict_anime_score(genre_list, anime_type, 'lr'))
print("Random Forest Prediction:", predict_anime_score(genre_list, anime_type, 'rf'))
print("GBT Prediction:", predict_anime_score(genre_list, anime_type, 'gbt'))

Linear Regression Prediction: 7.087673496089046
Random Forest Prediction: 7.135016389804815
GBT Prediction: 7.083338712701398


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Daftar genre dan tipe untuk dropdown
genre_list = ['Comedy', 'Action', 'Adventure', 'Fantasy', 'Drama', 'Sci-Fi', 'Shounen',
              'Romance', 'School', 'Slice of Life', 'Supernatural', 'Magic', 'Mecha',
              'Seinen', 'Historical', 'Kids', 'Ecchi', 'Shoujo', 'Mystery', 'Sports',
              'Military', 'Super Power', 'Music', 'Demons', 'Harem', 'Space', 'Parody',
              'Martial Arts', 'Horror', 'Game', 'Psychological', 'Police', 'Samurai',
              'Vampire', 'Thriller', 'Shoujo Ai', 'Josei', 'Shounen Ai', 'Cars', 'Dementia']
type_list = ['TV', 'Movie', 'OVA', 'Special', 'Unknown']

# Buat widgets
genre_select = widgets.SelectMultiple(
    options=genre_list,
    description='Genre:',
    disabled=False,
    layout={'width': '500px'}
)

type_select = widgets.Dropdown(
    options=type_list,
    description='Type:',
    disabled=False,
)

model_select = widgets.Dropdown(
    options=['Linear Regression', 'Random Forest', 'GBT'],
    description='Model:',
    disabled=False,
)

output = widgets.Output()

def on_predict_click(b):
    with output:
        output.clear_output()
        genres = list(genre_select.value)
        anime_type = type_select.value
        model_type = model_select.value.lower().replace(' ', '_')[:2]  # 'lr', 'rf', or 'gb'

        try:
            prediction = predict_anime_score(genres, anime_type, model_type)
            print(f"Predicted Score: {prediction:.2f}")
            print(f"\nInput parameters:")
            print(f"Genres: {', '.join(genres)}")
            print(f"Type: {anime_type}")
            print(f"Model: {model_select.value}")
        except Exception as e:
            print(f"Error occurred: {str(e)}")

predict_button = widgets.Button(description="Predict Score")
predict_button.on_click(on_predict_click)

# Display widgets
display(widgets.VBox([
    widgets.Label('Select Genres (multiple):'),
    genre_select,
    type_select,
    model_select,
    predict_button,
    output
]))